In [ ]:
%%capture
!pip install pandas scikit-learn numpy matplotlib torch
!kaggle competitions download titanic
!unzip -o titanic.zip

In [ ]:
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import PCA, KernelPCA
from sklearn.impute import KNNImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import numpy as np

# Model I: Birkenhead Model

To get things going, we can make an easy first pass model that operates on the knowledge that women and children were prioritized evacuation under the [Birkenhead drill](https://en.wikipedia.org/wiki/Women_and_children_first).  Simply put, if you are a woman or under the age of 13, the model will predict that the individual survives.

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train.sample(5)

In [ ]:
child_age = 12.0

# Keep sex and age columns
reduced = train[["Sex", "Age", "Survived"]].copy()
reduced.Sex = (reduced.Sex == "female").astype(int)
reduced.Age = (reduced.Age <= child_age).astype(int)
reduced["predicted"] = reduced.Sex | reduced.Age
train_acc = (reduced.predicted == reduced.Survived).mean()
sex_only = (reduced.Sex == reduced.Survived).mean()
print(f"Birkenhead drill train accuracy: {train_acc * 100:.2f}%")
print(f"Sex only train accuracy: {sex_only * 100:.2f}%")

# Write out the submission
bh_test = test[["PassengerId", "Sex", "Age"]].copy()
bh_test.Sex = (bh_test.Sex == "female").astype(int)
bh_test.Age = (bh_test.Age <= child_age).astype(int)
bh_test["Survived"] = bh_test.Sex | bh_test.Age
bh_test.to_csv("birkenhead.csv", columns=["PassengerId", "Survived"], index=False)

In [ ]:
#!kaggle competitions submit titanic -f birkenhead.csv -m "Naive Model"

Not bad!  For such a simple model we are nearly 80% accurate.  Adjusting for children didn't make much of a difference, but it did give us an extra half a percent or so.  

# Model II: Logistic Regression with Simple Imputation

Let's get all datapoints involved by using a simple linear model to classify survival.  We still have to deal with the case where data is missing from some columns, so let's apply the simplest imputation strategy we can derive: modal imputation.  We'll just fill all missing values with the most common value for that column and move on.  This will help establish a baseline for future approaches.

In [ ]:
# What columns lack entries?
train.isna().sum()

In [ ]:
def simple_data_prep(data: pd.DataFrame) -> pd.DataFrame:
    cleaned = data.copy(deep=True)
    cleaned.Sex = (cleaned.Sex == "female").astype(int)

    # Drop complex string fields like name, Cabin, and Ticket
    cleaned.drop(columns=["Name", "Cabin", "Ticket"], inplace=True)

    # Impute Age, Embarked
    cleaned.Age.fillna(cleaned.Age.median(), inplace=True)
    cleaned.Fare.fillna(cleaned.Fare.median(), inplace=True)
    cleaned.Embarked.fillna(cleaned.Embarked.mode().iloc[0], inplace=True)

    # Get dummy variables for Embarked
    cleaned = pd.get_dummies(cleaned, columns=["Embarked"])

    # Log fare to get rid of long tails
    cleaned.Fare = np.log10(cleaned.Fare + 1)

    # Normalize age
    cleaned.Age = cleaned.Age / cleaned.Age.max()
    return cleaned

In [ ]:
train_clean = simple_data_prep(train)
test_clean = simple_data_prep(test)
train_clean

In [ ]:
# Train using logistic regression
lgr = LogisticRegression(max_iter=1000, verbose=True)
x_cols = [x for x in train_clean.columns if x not in ["Survived", "PassengerId"]]
print(x_cols)
lgr.fit(train_clean[x_cols], y=train_clean.Survived)
train_acc = np.mean(lgr.predict(train_clean[x_cols]) == train_clean["Survived"]) * 100
print(f"Accuracy = {train_acc:.2f}")

test_clean["Survived"] = lgr.predict(test_clean[x_cols])
test_clean[["PassengerId", "Survived"]].to_csv("simple_lgr.csv", index=False)

In [ ]:
#!kaggle competitions submit titanic -f simple_lgr.csv -m "Simple imputation and logistic regression"

# Better Imputation

Before trying out any more sophisticated models, let's try to clean up our data to get the most out of what we have.

## Breaking Down String Fields

String fields like Cabin, Ticket, Name, and Embarked cannot be inputs to most models at this point and must be parsed into numerical fields.  Fortunately, these fields have lots of rich structure that embed information about the passenger.  This section will deal with breaking information out of these fields into subfields that we can use to impute missing fields and ultimately train a model on.

### Passenger Names

Fortunately, passenger's names on the ticket reveal quite a bit of information about their status, which is highly correlated to their age.  For example:

- "Master" is a honorific for young men or boys, meaning the age is likely under 18 years.  
- Married women are given "Mrs" while unmarried women or girls are "Miss"
- Some married women have their husbands name in parentheses. Some do not, possibly implying a widow?  Widows tend to be older 

These features in the name field can be extracted to binary variables which can be used to impute ages.

### Tickets

Some tickets are simply numbered, but others have letters or other encodings beforehand.  To extract meaningful information from the ticket field, split out any letters, or what we will call "ticket modifiers" and numbers, or "ticket numbers".  Modifiers will be mapped to an integer encoding with 0 being no characters in front of a ticket number.  Tickets with only letters will have ticket number 0.

### Cabin Number

The least populated field, cabin number seems to encode a number of things - the deck of the ship, the room number, and possibly the number of cabins this passenger has purchased.  Some cabin entries only have a deck associated with them and some have multiple entries.  We can break this field down into three variables: 
- An integer encoding for the deck the passenger was assigned to
- The cabin number or first cabin number in the first cabin entry
- The number of cabin numbers or spaces in the cabin entry

In [ ]:
# Determine which fields have missing entries in both datasets
train.isna().sum()

In [ ]:
test.isna().sum()

In [ ]:
def process_names(data: pd.DataFrame) -> pd.DataFrame:
    """Process passenger names into several columns that represent title information in a categorical
    variable

    Args:
        data (pd.DataFrame): A dataframe with column "Name"

    Returns:
        pd.DataFrame: A DataFrame with new columns derived from "Name"
    """
    modified = data.copy(deep=True)

    # Develop columns based on the passenger's name
    modified["master"] = (data.Name.str.contains("master", case=False)).astype(int)
    modified["mrs"] = (data.Name.str.contains("Mrs", case=True)).astype(int)
    modified["miss"] = (data.Name.str.contains("Miss", case=True)).astype(int)
    modified["widow"] = (modified["mrs"] & ~data.Name.str.contains("\(")).astype(int)
    modified["mr"] = (data.Name.str.contains("Mr.", case=True)).astype(int)
    modified["surname"] = modified.Name.apply(
        lambda x: x.split(" ")[0].replace(",", "")
    )
    unique_names = modified.surname.unique()
    surname_map = dict(zip(unique_names, range(len(unique_names))))
    modified.surname = modified.surname.apply(lambda x: surname_map[x])
    modified.drop(columns=["Name"], inplace=True)
    return modified

In [ ]:
def ticket_to_number(ticket):
    if isinstance(ticket, (int, float)):
        return int(ticket)
    else:
        no_letters = "".join(filter(str.isdigit, ticket))
        if len(no_letters):
            return int(no_letters)
        else:
            return 0


def process_ticket(data: pd.DataFrame) -> pd.DataFrame:
    modified = data.copy(deep=True)
    # Develop columns based on the ticket description
    ticket_modifiers = {""}
    for ticket in data.Ticket.tolist():
        # Take only letters, some modifiers vary in capitalization, so lower before adding
        # to our set
        modifier = "".join(filter(str.isalpha, ticket)).lower()
        ticket_modifiers.add(modifier)
    ticket_modifiers = dict(zip(ticket_modifiers, range(len(ticket_modifiers))))

    modified["ticket_modifier"] = modified.Ticket.apply(
        lambda x: ticket_modifiers["".join(filter(str.isalpha, x)).lower()]
    )
    modified["ticket_number"] = modified.Ticket.apply(ticket_to_number)
    modified["ticket_digits"] = modified["ticket_number"].apply(lambda x: len(str(x)))
    modified.drop(columns=["Ticket"], inplace=True)
    return modified

In [ ]:
def get_number(cabin_entry):
    """Extract a cabin number if one exists or take the mean of all numbers found"""
    if isinstance(cabin_entry, (int, float)):
        return int(cabin_entry)
    else:
        entries = cabin_entry.split(" ")
        numbers = ["".join(filter(str.isdigit, x)) for x in entries]
        numbers = [int(x) for x in numbers if len(x)]
        if len(numbers) == 0:
            return np.nan
        else:
            return sum(numbers) / len(numbers)


def process_cabin(data: pd.DataFrame) -> pd.DataFrame:
    modified = data.copy(deep=True)
    # Develop columns based on the cabin
    with_cabins = modified[~modified.Cabin.isna()]
    modified.loc[~modified.Cabin.isna(), "cabin_entries"] = with_cabins.Cabin.apply(
        lambda x: len(str(x).split(" "))
    )

    # Develop an encoding for the deck level, leave missing ones as NA
    for idx, level in enumerate(["a", "b", "c", "d", "e", "f"]):
        modified.loc[
            ~modified.Cabin.isna() & with_cabins.Cabin.str.contains(level, case=False),
            "cabin_level",
        ] = (
            idx + 1
        )
    modified.loc[with_cabins.index, "cabin_number"] = with_cabins.Cabin.apply(
        get_number
    )
    modified.drop(columns=["Cabin"], inplace=True)
    return modified

In [ ]:
def process_string_fields(data: pd.DataFrame) -> pd.DataFrame:
    modified = data.copy(deep=True)

    # Process Sex to be binary
    modified.Sex = (modified.Sex == "female").astype(int)
    # Get dummy variables for embarked
    modified = pd.get_dummies(modified, columns=["Embarked"])
    modified = process_names(modified)
    modified = process_cabin(modified)
    modified = process_ticket(modified)
    return modified

# Imputation Methods

There are many different approaches to imputing missing age and cabin values.  We'll look into two different approaches:

1. Manually determining groups of individuals and assigning a value based on their statistical distribution
2. Using a nearest-neighbor method to draw conclusions about similarities between individuals and propagating known values

Option 1 may be more heavy handed, but will allow us to prioritize assumptions based on name or other string fields to determine age.  Option 2 would be better suited for inferring cabin, where it's less obvious what the value may be, but it's clear what variables may influence the missing quantity.

In [ ]:
def impute_age_groups(data: pd.DataFrame) -> pd.DataFrame:
    modified = data.copy(deep=True)
    is_ageless = modified.Age.isna()
    male = modified.Sex == 0
    # Find masters in the group and impute age
    is_master = modified.master == 1
    modified.loc[is_ageless & is_master, "Age"] = modified.loc[
        is_master, "Age"
    ].median()

    # Age for unmarried men (not master title) and women that are part of families
    in_family = (modified.Parch > 0) & (modified.SibSp > 0)
    is_miss = modified.miss == 1
    modified.loc[is_ageless & is_miss & in_family, "Age"] = modified.loc[
        is_miss & in_family, "Age"
    ].median()
    modified.loc[is_ageless & male & in_family, "Age"] = modified.loc[
        male & in_family, "Age"
    ].median()

    # Children travelling with a nanny (SibSp == 0, Parch == 0)
    with_nanny = (modified.Parch == 0) & (modified.SibSp > 0)
    modified.loc[is_ageless & with_nanny, "Age"] = modified.loc[
        with_nanny, "Age"
    ].median()

    # Age for married women based on widow status
    widow = modified.widow == 1
    modified.loc[is_ageless & widow, "Age"] = modified.loc[widow, "Age"].median()

    # Married women
    married_woman = modified.mrs & ~widow
    modified.loc[is_ageless & married_woman, "Age"] = modified.loc[
        married_woman, "Age"
    ].median()

    # Single men and women (SibSp and Parch is 0).  Treat these two differently as I could imagine younger men probably travelled alone more often
    # than younger women
    alone = (modified.SibSp == 0) & (modified.Parch == 0)
    single_man = (modified.Sex == 0) & alone
    single_woman = (modified.Sex == 1) & alone & modified.miss
    modified.loc[is_ageless & single_man, "Age"] = modified.loc[
        single_man, "Age"
    ].median()
    modified.loc[is_ageless & single_woman, "Age"] = modified.loc[
        single_man, "Age"
    ].median()

    # Married men (Mr. and SibSp > 0 and Parch = 0)
    married_men_no_kids = (
        (modified.Sex == 0) & (modified.SibSp == 1) & (modified.Parch == 0)
    )
    modified.loc[is_ageless & married_men_no_kids, "Age"] = modified.loc[
        married_men_no_kids, "Age"
    ].median()

    # Cover all other conditions
    is_still_ageless = modified.Age.isna()
    modified.loc[is_still_ageless, "Age"] = modified.Age.median()

    # Check we've imputed age for all passengers
    assert len(modified[modified.Age.isna()]) == 0
    return modified

In [ ]:
def impute_cabin(data: pd.DataFrame) -> pd.DataFrame:
    modified = data.copy(deep=True)
    imputer = KNNImputer()
    out = imputer.fit_transform(modified)
    modified = out
    return modified

In [ ]:
def preprocess(data):
    modified = data.copy(deep=True)
    modified = process_string_fields(modified)
    # Impute the fare 
    modified.Fare.fillna(modified.Fare.mean())
    
    modified = impute_age_groups(modified)
    modified[modified.columns] = impute_cabin(modified)
    return modified

In [ ]:
def normalize(data):
    modified = data.copy(deep=True)

    # Normalize fair, ticket number by log10
    modified.Fare = np.log10(modified.Fare + 1)
    modified.ticket_number = np.log10(modified.ticket_number + 1)
    # Normalize all columns
    for column in modified.columns:
        min_max = (modified[column].max() - modified[column].min())
        if min_max != 0:
            modified[column] = (modified[column] - modified[column].min()) / min_max
    return modified

In [ ]:
train_clean = preprocess(train)
test_clean = preprocess(test)

train_norm = normalize(train_clean)
test_norm = normalize(test_clean)

In [ ]:
t_train, t_val = train_test_split(train_norm, test_size=.2)
classifiers = [LogisticRegression(max_iter=500), SVC()]
train_x = t_train[[x for x in t_train.columns if x not in ['Survived', 'PassengerId']]]
train_y = t_train['Survived']
val_x = t_val[[x for x in t_val.columns if x not in ['Survived', 'PassengerId']]]
val_y = t_val['Survived']

for classifier in classifiers:
    classifier.fit(train_x, train_y)
    train_acc = np.mean(classifier.predict(train_x) == train_y) * 100
    val_acc = np.mean(classifier.predict(val_x) == val_y) * 100
    print(train_acc)
    print(val_acc)

In [ ]:
train_cols = [x for x in train_norm.columns if x not in ['Survived', 'PassengerId']]
train_x = train_norm[train_cols]
train_y = train_norm['Survived']
best_classifier = LogisticRegression()

best_classifier.fit(train_x, train_y)
train_acc = np.mean(best_classifier.predict(train_x) == train_y) * 100
print(train_acc)

test_x = test_norm[train_cols]
test['Survived'] = best_classifier.predict(test_x)
test['Survived'] = test['Survived'].astype(int)
test[['PassengerId', 'Survived']].to_csv('grouped_age_cabin_lr.csv', index=False)

In [ ]:
!kaggle competitions submit titanic -f grouped_age_cabin_lr.csv -m "Age imputation by manually developed groups, cabin imputation, LR"

In [ ]:
new_stuff = pd.read_csv('grouped_age_cabin_lr.csv')
    new_stuff.Survived

In [ ]:
np.mean(new_stuff.Survived == old_stuff.Survived)

In [ ]:
new_stuff